In [11]:
# ==================
# Library
# ==================
import warnings
warnings.simplefilter('ignore')
import math
import pandas as pd
import numpy as np
import lightgbm as lgb
import time
import datetime
import gc
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold,GroupKFold
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from contextlib import contextmanager
import logging
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import cohen_kappa_score, mean_squared_error
from sklearn.metrics import log_loss
import sys
import time
import feather
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import pickle
%matplotlib inline
from sklearn.metrics import average_precision_score

sys.path.append("../src/")
from logger import setup_logger, LOGGER
from trainer import train_lgbm
from util_tool import reduce_mem_usage
pd.set_option('display.max_columns', 300)

In [21]:
# ==================
# Constant
# ==================
ex = "001"
TRAIN_PATH = "../input/train.csv"
TEST_PATH = "../input/test.csv"
USER_PATH = "../input/user_x_anime.csv"
SUB_PATH = "../input/sample_submission.csv"
SAVE_OOF_PATH = f"../output/exp/ex{ex}_oof.npy"
SAVE_TEST_SUB_PATH = f"../output/exp/ex{ex}_test_sub.csv"
LOGGER_PATH = f"../output/exp/ex_{ex}.txt"

In [13]:

# ===============
# Settings
# ===============

SEED = 0
N_SPLITS = 5
SHUFFLE = True
LGBM_PARAMS = {'num_leaves': 32,
               'min_data_in_leaf': 64,
               'objective': 'regression',
               'max_depth': -1,
               'learning_rate': 0.05,
               "boosting": "gbdt",
               "bagging_freq": 1,
               "bagging_fraction": 0.8,
               "bagging_seed": SEED,
               "verbosity": -1,
              'reg_alpha': 0.1,
              'reg_lambda': 0.3,
              'colsample_bytree': 0.7,
              'metric':"rmse",
              'num_threads':32,
         }

LGBM_FIT_PARAMS = {
    'num_boost_round': 10000,
    'early_stopping_rounds': 200,
    'verbose_eval': 200,
}

load_feature = ["../output/fe/fe001.feather"]

In [14]:
# ====================
# Function
# ====================

def calc_loss(y_true, y_pred):
    return  np.sqrt(mean_squared_error(y_true, y_pred))


@contextmanager
def timer(name):
    t0 = time.time()
    yield 
    LOGGER.info(f'[{name}] done in {time.time() - t0:.0f} s')

In [19]:
LOGGER = logging.getLogger()
FORMATTER = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
setup_logger(out_file=LOGGER_PATH)

2021-08-17 10:23:07,748 - INFO - logger set up


<RootLogger root (DEBUG)>

In [15]:
# ====================
# Main
# ====================
train_raw = pd.read_csv(TRAIN_PATH)
y = train_raw["Score"]
df = None
for i in load_feature:
    if df is not None:
        _df = pd.read_feather(i)
        df  = pd.concat([df, _df], axis=1)
    else:
        df = pd.read_feather(i)
train = df.iloc[:len(train_raw)]
test = df.iloc[len(train_raw):].reset_index(drop=True)

In [16]:
train

,Episodes,Members,Favorites,Watching,Completed,On-Hold,Dropped,Plan to Watch
0,26.0,1251960,61971,105808,718161,71513,26678,329800
1,24.0,214499,4101,11909,81145,11901,11026,98518
2,178.0,141832,3124,11235,76881,12905,12516,28295
3,12.0,3648,16,170,1333,145,296,1704
4,1.0,566538,9555,8258,434086,3491,2174,118529
...,...,...,...,...,...,...,...,...
4995,1.0,437,6,6,396,1,2,32
4996,1.0,18717,49,1436,12325,172,120,4664
4997,NaN,1575,8,908,1,18,22,626
4998,NaN,5392,19,1053,1,178,109,4051


In [20]:
with timer("lightgbm"):
    kf = KFold(n_splits=N_SPLITS,random_state=SEED, shuffle=SHUFFLE)
    y_oof = np.empty([len(train),])
    y_test = []
    drop_cols = []
    features = list(train.columns)
    features = [i for i in features if i not in drop_cols]
    feature_importances = pd.DataFrame()
    categorical_features = []
    for fold, (train_idx, valid_idx) in enumerate(kf.split(train,y)):
        print('Fold {}'.format(fold + 1))
        with timer(f"fold {fold}"):
            x_train, y_train = train.iloc[train_idx][features], y.iloc[train_idx]
            x_val, y_val =train.iloc[valid_idx][features], y.iloc[valid_idx]
            print("train:",len(x_train))

            y_pred_valid, y_pred_test, valid_loss, importances, best_iter,_ = train_lgbm(
                        x_train, y_train, x_val, y_val,test[features],
                        categorical_features=categorical_features,
                        feature_name=features,
                        fold_id=fold,
                        lgb_params=LGBM_PARAMS,
                        fit_params=LGBM_FIT_PARAMS,
                        loss_func=calc_loss,
                        calc_importances=True
                    )

            y_oof[valid_idx] = y_pred_valid
            score = calc_loss(y[valid_idx], y_pred_valid)
            LOGGER.info(f'Fold{fold}:CV={score}')
            y_test.append(y_pred_test)
            feature_importances = pd.concat([feature_importances, importances], axis=0, sort=False)

    score = calc_loss(y, y_oof)
    np.save(SAVE_OOF_PATH, y_oof)
    LOGGER.info(f'CV={score}')

Fold 1
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.373619	valid_1's rmse: 0.441249
[400]	training's rmse: 0.322419	valid_1's rmse: 0.442235


2021-08-17 10:23:12,053 - INFO - Fold0:CV=0.4396559710880034
2021-08-17 10:23:12,068 - INFO - [fold 0] done in 3 s


Early stopping, best iteration is:
[332]	training's rmse: 0.337047	valid_1's rmse: 0.439656
Fold 2
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.372257	valid_1's rmse: 0.455016
[400]	training's rmse: 0.321967	valid_1's rmse: 0.453274


2021-08-17 10:23:14,812 - INFO - Fold1:CV=0.4522081846496576
2021-08-17 10:23:14,816 - INFO - [fold 1] done in 3 s


Early stopping, best iteration is:
[345]	training's rmse: 0.333247	valid_1's rmse: 0.452208
Fold 3
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.367526	valid_1's rmse: 0.481774
[400]	training's rmse: 0.318034	valid_1's rmse: 0.48248


2021-08-17 10:23:17,214 - INFO - Fold2:CV=0.48006439015009467
2021-08-17 10:23:17,218 - INFO - [fold 2] done in 2 s


Early stopping, best iteration is:
[270]	training's rmse: 0.347407	valid_1's rmse: 0.480064
Fold 4
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.373057	valid_1's rmse: 0.446844
[400]	training's rmse: 0.320856	valid_1's rmse: 0.449784
Early stopping, best iteration is:
[231]	training's rmse: 0.363069	valid_1's rmse: 0.446424


2021-08-17 10:23:19,213 - INFO - Fold3:CV=0.4464239253394724
2021-08-17 10:23:19,234 - INFO - [fold 3] done in 2 s


Fold 5
train: 4000
Training until validation scores don't improve for 200 rounds
[200]	training's rmse: 0.366018	valid_1's rmse: 0.487382
[400]	training's rmse: 0.317309	valid_1's rmse: 0.483197
[600]	training's rmse: 0.283187	valid_1's rmse: 0.482793
Early stopping, best iteration is:
[572]	training's rmse: 0.28744	valid_1's rmse: 0.481678


2021-08-17 10:23:24,125 - INFO - Fold4:CV=0.4816783746266547
2021-08-17 10:23:24,140 - INFO - [fold 4] done in 5 s
2021-08-17 10:23:24,153 - INFO - CV=0.460338964582868
2021-08-17 10:23:24,157 - INFO - [lightgbm] done in 15 s


In [22]:
y_test_sub =  np.mean(y_test,axis=0)
y_test_sub = np.expm1(y_test_sub)
sub = pd.read_csv(SUB_PATH)

In [24]:
sub["score"] = y_test_sub
sub.to_csv(SAVE_TEST_SUB_PATH,index=False)

In [27]:
feature_importances.groupby(by="feature").mean().sort_values(by="gain",ascending=False)

,gain,split,fold
feature,,,
Favorites,10964.506228,1083.6,2.0
Dropped,2855.405705,2058.6,2.0
Plan to Watch,2339.983987,1329.2,2.0
On-Hold,1896.083180,1645.6,2.0
Members,1806.226883,1257.6,2.0
Completed,1399.416945,1440.6,2.0
Watching,1212.345018,1417.4,2.0
Episodes,847.673964,617.4,2.0
